# LDA主题提取实例

## 导入数据（观察是否正确）

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('datascience.csv',encoding='gb18030')

注意上面encoding的格式编码，不是一般的utf-8,所以gb18030这个编码格式的中文文件是可以直接在excel中打开的

In [3]:
df.head(5)

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会??借“创响中国”贵安站巡回接力活动暨2016贵安...


In [4]:
df.shape

(1024, 3)

## 进行分词

In [5]:
import jieba

注意，这次要处理的，是分散的1000多条语句，而不是一整块文本，所以可以使用函数，来一句一句的处理

In [6]:
def zh_word_cut(sentence):
    return " ".join(jieba.cut(sentence))

+ join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
+ str.join(sequence)
+ 这里的空格.join 意思就是把后面分出来字符串用空格连接成一个新的字符串
+ 这样** 返回来的就不是字符型列表了，处理起来会更方便，get**

> 使用上面那个函数需要用循环来不断调用，来处理数据框（df）里的全部文本，但是这样比较慢，可以使用更高效的apply函数，如下

In [7]:
df['content_cutted']=df.content.apply(zh_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\shanshan\AppData\Local\Temp\jieba.cache
Loading model cost 1.399 seconds.
Prefix dict has been built succesfully.


In [8]:
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会 ? ? 借 “ 创响 中国 ”...
Name: content_cutted, dtype: object

+ 可以看到，已经分词成功，但是很明显，有错误
+ 例如： 大数据 被分成了 大 数据

## 文本向量化

单词之间已经被空格分开了，形式就像英文一样，这时候可以对文本进行向量化（也就是变成数值形式，方便处理）

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

CountVectorizer与TfidfVectorizer，这两个类都是特征数值计算的常见方法。
+ 对于每一个训练文本，CountVectorizer只考虑每种词汇在该训练文本中出现的频率，
+ 而TfidfVectorizer除了考量某一词汇在当前训练文本中出现的频率之外，同时关注包含这个词汇的其它训练文本数目的倒数。
+ 相比之下，训练文本的数量越多，TfidfVectorizer这种特征量化方式就更有优势。

>+ 词集模型：单词构成的集合，每个单词只出现一次。 sow set of word
>+ 词袋模型：把每一个单词都进行统计，同时计算每个单词出现的次数。 bow bag of word
>+ Tf-IDF模型

由于这些文本可能会含有大量的词汇，我们不希望处理所有词汇，所以限定一下，只提取1000个最重要的特征关键词，然后停止
+ 看了下数据，content是一个微信公众号内容的完整全文。。所以很大，所以只要抽1k个词

In [10]:
n_features=1000

In [11]:
tf_vectorizer=CountVectorizer(strip_accents='unicode',
                             max_features=1000,
                             stop_words='english',
                             max_df=0.5,
                             min_df=10)

> strip_accents(重音)：{'ascii'，'unicode'，None}
    + 在预处理步骤中删除重音符号。 'ascii'是一种快速方法，仅适用于具有直接ASCII映射的字符。 'unicode'是一个适用于任何字符的稍慢的方法。 无（默认）什么也不做。
> stop_words：string {'english'}，列表或无（默认）
    + 如果'英语'，则使用内置的英语停用词表。
    + 也可以给出自定义的停用词列表，那么所有列表中的词都将从结果标记中删除。 仅适用于分析器=='单词'analyzer == 'word' 单词级别的分析，不是句子级别。
    + 如果为None，则不会使用停用词。 可以将max_df设置为范围[0.7,1.0）中的一个值，基于内部语料库文档的词频来过滤停用词
    
> max_df：[0.0，1.0]范围内的float或int，默认值= 1.0
    + 在构建词汇时忽略文档频率严格高于给定阈值（特定语料库的停用词）的词语。 
    + 如果为float，则该参数表示文档的比例，若是整数，则表示绝对计数，词出现的次数上限。 
    + 如果vocabulary参数是not None，则忽略此参数。

>min_df：[0.0，1.0]范围内的浮点数float或int，默认值= 1
    + 在构建词汇时忽略文档频率严格低于给定阈值的词语。 这个值在文献中也被称为截断值。
    + 如果为float，则该参数表示文档的比例，若是整数，则表示绝对计数，词出现的次数下限。
    + 如果vocabulary参数是not None，则忽略此参数。

>max_features：int或None，default = None
    + 如果不是None，则建立一个词汇表，仅考虑整个语料库中按词频排序的顶级max_features。
    + 如果vocabulary参数是not None，则忽略此参数

所以上面只考虑词频最高的前1000个词，这属于词袋模型
<br>** 向量转换的同时只按照频率提取前1000频率的词 **

In [12]:
tf=tf_vectorizer.fit_transform(df.content_cutted)

> sklearn.feature_extraction.text.CountVectorizer.fit_transform(raw_documents, y=None)[source]
  学习词汇词典并返回术语 - 文档矩阵。fit_transform相当于fit之后紧接着进行transform，只是更高效一点
    + 参数：
    + raw_documents：可迭代的，一个可以产生str，unicode或file对象的迭代器。
    + 返回：
    + X：数组，[n_samples，n_features]，文档术语矩阵。

In [13]:
print(tf[7])

  (0, 17)	1
  (0, 332)	1
  (0, 689)	2
  (0, 439)	1
  (0, 209)	1
  (0, 165)	1
  (0, 941)	1
  (0, 10)	1
  (0, 538)	1
  (0, 437)	1
  (0, 297)	1
  (0, 544)	1
  (0, 753)	1
  (0, 99)	1
  (0, 78)	1
  (0, 816)	1
  (0, 625)	1
  (0, 134)	1
  (0, 384)	1
  (0, 93)	1
  (0, 572)	1
  (0, 798)	1
  (0, 502)	1
  (0, 0)	2
  (0, 31)	1
  :	:
  (0, 671)	1
  (0, 575)	1
  (0, 992)	1
  (0, 110)	2
  (0, 514)	1
  (0, 517)	1
  (0, 871)	1
  (0, 580)	1
  (0, 265)	1
  (0, 187)	2
  (0, 361)	2
  (0, 875)	1
  (0, 162)	4
  (0, 275)	1
  (0, 410)	1
  (0, 366)	1
  (0, 686)	1
  (0, 926)	2
  (0, 556)	1
  (0, 117)	4
  (0, 486)	1
  (0, 716)	4
  (0, 847)	1
  (0, 748)	1
  (0, 212)	1


可以使用类似 print(tf[1]) 这样的语句看一下向量化之后的结果
+ 输入tf.shape
    + 返回(1024, 1000) 
    + 也就是1024个记录，其中每个记录1000个关键词？但是很明显，有的句子很短
+ 输入tf[1]
    + 返回 <1x1000 sparse matrix of type '<class 'numpy.int64'>'with 161 stored elements in Compressed Sparse Row format>,
    + 所以返回的是一个1行，1000列的稀疏矩阵，其中每个元素的类型是 以压缩稀疏行格式存储的
    + 注意观察返回数据形式，如下，可以发现，元组其实表示的是序号（0行，因为tf中每个元素是一个行向量，所以行号都为0，×列，因为提取词频最高的1k词，所以列号从0到999），后面的 1 是data，
 + print(tf[11])
 + (0, 276)	1
 + (0, 402)	1
 + (0, 760)	1
 + ** (0, 999)	1**
 + (0, 330)	1
 + (0, 588)	2
 + (0, 257)	2
 + (0, 189)	4
 + (0, 666)	15<br><br>
 + print(tf[7])
 + (0, 502)	1
 + **(0, 0)	2**
 + (0, 31)	1
 +  :	:
+ 输入type(tf[1])
    + 返回scipy.sparse.csr.csr_matrix ，所以这个就是Compressed Sparse Row 压缩稀疏行矩阵
+ tf[1].shape
    + 返回(1, 1000)，所以tf中的每个元素都有1k元素
+ tf.dtype
    + 返回dtype('int64')，所以元素的元素都是整数类型(数字)，已经转为向量了
+ 输入tf[1][0,2]
    + 因为tf中每个元素都是矩阵，所以可以使用这样的方式访问矩阵中的元素
+ sorted(tf.data,reverse=True)
    + 从返回的值中可以看到，是从1到319的词
    + 虽然指明要词频前1k的词，但是在构建模型的时候，又指定了max_df=0.5,min_df=10，选取了出现次数10次以上，同时最大出现频率是1000多个文档的50%，一共是1024个记录，也就是说那个词语最多在512个记录中出现
+ 特征词确定之后，就可以使用模型对这些特征词进行分析了

## 主题提取

**应用LDA方法，需要人为设定主题的数量**
+ 指定（或者叫瞎猜）主题个数是必须的。
    + 如果你只需要把文章粗略划分成几个大类，就可以把数字设定小一些；
    + 相反，如果你希望能够识别出非常细分的主题，就增大主题个数。
+ 对划分的结果，如果你觉得不够满意，可以通过继续迭代，调整主题数量来优化

可以先设定5个分类，试试

https://www.cnblogs.com/pinard/p/6908150.html 这个博客有更详细的函数解释

In [14]:
n_topics=5

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

In [16]:
lda=LatentDirichletAllocation(n_topics=5,max_iter=50,
                              learning_method='online',
                              learning_offset=50.,
                              random_state=0)

> max_iter : integer, optional (default=10)最大迭代次数

> random_state : int, RandomState instance or None, optional (default=None)
    + 如果是整数，则random_state代表被随机数产生器使用的种子编号
    + 如果是RandomState实例，random_state就代表随机数产生器
    + 若为None，则随机数产生器就是所使用的RandomState实例
    + by `np.random`.
>learning_method : 'batch' | 'online', default='online'
    用于更新_component的方法，只用于‘fit’方法
    通常当数据集很大时，在线（online）更新将比批量(batch)更新快很多
    在sklearn0.20版本中，默认的方式将换为batch
    有效选项：
    + batch:批量变分贝叶斯方法，在每一次EM更新中使用所有的训练数据, 旧的`components_`将在每次迭代中被覆盖。
    + online:在线变分贝叶斯方法。 在每次EM更新中，使用 最小批量的训练数据来更新``components_``` 增量可变。 学习速度由 ``learning_decay``和``learning_offset``参数控制
    
    
> learning_decay（衰减） : float, optional (default=0.7)
    是控制在线学习方法‘学习率‘的参数.为了保证渐进收敛值应该设置在(0.5,1.0]之间，当值为0.0并且batch_size是‘n_sample’（也就是整个样本数）
    此时的更新方法和batch learning一样。用术语来说，叫kappa

> learning_offset（补偿，抵消） : float, optional (default=10.)
    一个用于降低在线学习早起迭代的（正）参数，应该大于1.0。用术语来说，称为tau_0。
     用来减小前面训练样本批次对最终模型的影响。

online learning强调的是学习是实时的，流式的，每次训练不用使用全部样本，而是以之前训练好的模型为基础，每来一个样本就更新一次模型，这种方法叫做OGD（online gradient descent）。这样做的目的是快速地进行模型的更新，提升模型时效性。

online learning其实细分又可以分为batch模式和delta模式。batch模式的时效性比delta模式要低一些。分析一下batch模式，比如昨天及昨天的数据训练成了模型M，那么今天的每一条训练数据在训练过程中都会更新一次模型M，从而生成今天的模型M1。

而batch learning或者叫offline learning强调的是每次训练都需要使用全量的样本，因而可能会面临数据量过大的问题。后面要讨论的批量梯度下降法（BGD）和随机梯度下降法（SGD）都属于batch learning或者offline learning的范畴。

batch learning一般进行多轮迭代来向最优解靠近。online learning没有多轮的概念，如果数据量不够或训练数据不够充分，通过copy多份同样的训练数据来模拟batch learning的多轮训练也是有效的方法。

+ 批量学习（batch learning），一次性批量输入给学习算法，可以被形象的称为填鸭式学习。
+ 在线学习（online learning），按照顺序，循序的学习，不断的去修正模型，进行优化。

In [19]:
lda.fit(tf)

D:\anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=5, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

上面这步可能会比较慢，不要着急，工作量比较大，需要一段时间
+ 这时候，就相当于把1000多篇文章向量化后的文章扔给了LDA，让他寻找主题
+ 跑完之后，只是返回了这个模型的各种参数，但是并没有看到主题，
    + 可以定义下列函数，来把每个主题的前若干个关键字显示出来

In [20]:
def print_top_words(model,feature_names,n_top_words):
    for topic_idx,topic in enumerate(model.components_):
        print('Topic #%d:'%topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))
    print()

函数定义好之后，暂定每个主题输出前20个关键词

In [21]:
n_top_words=20

In [22]:
tf_feature_names=tf_vectorizer.get_feature_names()
print_top_words(lda,tf_feature_names,n_top_words=n_top_words)

Topic #0:
系统 数据分析 使用 业务 数据库 管理 电子 存储 采集 产品 工具 工作 企业 用户 处理 相关 或者 平台 支持 项目
Topic #1:
这个 就是 可能 他们 如果 没有 自己 很多 什么 不是 但是 这样 现在 一些 因为 时候 已经 所以 非常 孩子
Topic #2:
学习 模型 算法 机器 方法 使用 神经网络 特征 训练 深度 分类 可视化 不同 这个 计算 函数 如果 网络 预测 结果
Topic #3:
企业 中国 互联网 行业 市场 服务 用户 平台 金融 2016 创新 城市 公司 产业 增长 国家 政府 经济 实现 10
Topic #4:
人工智能 领域 智能 学习 机器人 公司 机器 人类 深度 研究 识别 医疗 未来 系统 目前 已经 语音 服务 计算机 工业



+ 在这5个主题里，可以看出主题0主要关注的是数据科学中的算法和技术，而主题4显然更注重数据科学的应用场景
+ 只看文字不够直观，可以绘图看看，可视化

In [23]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

D:\anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.170460 -0.001113       1        1  26.233888
1     -0.072788  0.115864       2        1  25.021653
2     -0.173405  0.005991       3        1  18.321243
0     -0.022152 -0.178953       4        1  16.863127
4      0.097885  0.058210       5        1  13.560089, topic_info=     Category         Freq    Term        Total  loglift  logprob
term                                                             
123   Default  2789.000000    人工智能  2789.000000  30.0000  30.0000
366   Default  4217.000000      学习  4217.000000  29.0000  29.0000
587   Default  1456.000000      智能  1456.000000  28.0000  28.0000
618   Default  1116.000000     机器人  1116.000000  27.0000  27.0000
992   Default  2007.000000      领域  2007.000000  26.0000  26.0000
781   Default  1853.000000      算法  1853.000000  25.0000  25.0000
617   Default  2194.000000      机器  2194.000000  24.0000  24.0000
647   Default  1779.000000      模型  1779.000000  23.0000  23.0000
931   Default  2250.000000      这个  2250.000000  22.0000  22.0000
149   Default  3205.000000      企业  3205.000000  21.0000  21.0000
680   Default  1343.000000      深度  1343.000000  20.0000  20.0000
126   Default  1022.000000      人类  1022.000000  19.0000  19.0000
194   Default  2438.000000      公司  2438.000000  18.0000  18.0000
758   Default   914.000000    神经网络   914.000000  17.0000  17.0000
403   Default  1787.000000      就是  1787.000000  16.0000  16.0000
91    Default  1754.000000      中国  1754.000000  15.0000  15.0000
558   Default   837.000000     数据库   837.000000  14.0000  14.0000
787   Default  1999.000000      系统  1999.000000  13.0000  13.0000
721   Default   728.000000      电子   728.000000  12.0000  12.0000
85    Default  1131.000000      业务  1131.000000  11.0000  11.0000
574   Default  1529.000000      方法  1529.000000  10.0000  10.0000
167   Default  2239.000000      使用  2239.000000   9.0000   9.0000
556   Default  1622.000000    数据分析  1622.000000   8.0000   8.0000
782   Default  1367.000000      管理  1367.000000   7.0000   7.0000
137   Default  1330.000000      他们  1330.000000   6.0000   6.0000
694   Default   881.000000      特征   881.000000   5.0000   5.0000
957   Default   746.000000      采集   746.000000   4.0000   4.0000
364   Default   742.000000      存储   742.000000   3.0000   3.0000
961   Default   986.000000      金融   986.000000   2.0000   2.0000
362   Default  1657.000000      如果  1657.000000   1.0000   1.0000
...       ...          ...     ...          ...      ...      ...
126    Topic5   635.830359      人类  1022.728698   1.5227  -4.4902
329    Topic5   111.758620      地图   180.505959   1.5186  -6.2288
504    Topic5    90.819498      报道   147.366420   1.5140  -6.4362
112    Topic5   144.373065      交互   235.052118   1.5106  -5.9727
549    Topic5   167.255445      教授   272.824331   1.5087  -5.8256
987    Topic5    91.565320      革命   149.590592   1.5072  -6.4281
409    Topic5   329.160830      工业   539.280137   1.5044  -5.1486
35     Topic5   197.045768  google   326.435276   1.4932  -5.6617
888    Topic5   491.830581      识别   869.634078   1.4281  -4.7470
680    Topic5   623.783712      深度  1343.477672   1.2308  -4.5093
617    Topic5   871.858951      机器  2194.848660   1.0748  -4.1745
194    Topic5   910.860098      公司  2438.701722   1.0132  -4.1307
872    Topic5   348.670109     计算机   728.498403   1.2612  -5.0910
730    Topic5   406.054478      目前   965.571549   1.1318  -4.9386
366    Topic5  1135.970751      学习  4217.150102   0.6864  -3.9099
612    Topic5   458.274863      未来  1243.337120   1.0000  -4.8176
749    Topic5   556.254582      研究  1724.594534   0.8665  -4.6239
761    Topic5   306.838351      科技   767.397871   1.0814  -5.2188
418    Topic5   391.826571      已经  1507.892158   0.6504  -4.9743
787    Topic5   446.951935      系统  1999.224708   0.5000  -4.8427
575    Topic5   327.775458      方面  1056.971254   0.8272  -5.1528
439    Topic5  

+ 上面的那个pyLDAvis是专门用于进行LDA的可视化的，也可以和gensim等里面的LDA模型进行交互,但是有个小bug，就是想要让上述图片显示出来，必须翻墙（我的还可以用，开心，带的动）
+ 我们规定的是5个主题，所以上面5个圈
+ 右边横向柱状图中红色代表的是每个关键词在当前主题（左边的圈）下的概率，所以这里的主题表示是词的概率分布，并没有明确的一句话总结的主题，不像语文
<br>** 接下来试试主题数目为10的情况**

In [24]:
n_topics=10

In [25]:
lda=LatentDirichletAllocation(learning_method='online',
                             learning_offset=50.,
                             random_state=0,
                             n_topics=n_topics,
                             max_iter=50)

In [26]:
lda.fit(tf)

D:\anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [27]:
print_top_words(lda,tf_feature_names,n_top_words)

Topic #0:
系统 存储 使用 数据库 业务 处理 工作 采集 hadoop 数据仓库 查询 支持 项目 开发 架构 sql 计算 数据分析 设备 平台
Topic #1:
人工智能 学习 机器 人类 机器人 孩子 研究 深度 智能 计算机 领域 已经 能力 未来 工作 系统 他们 能够 可能 教育
Topic #2:
学习 算法 模型 方法 神经网络 机器 特征 训练 深度 分类 使用 函数 计算 预测 参数 不同 样本 结果 网络 这个
Topic #3:
企业 管理 服务 平台 建设 互联网 政府 创新 资源 金融 实现 安全 行业 建立 开放 社会 信息化 智慧 国家 能力
Topic #4:
可视化 使用 data 图表 工具 http 检索 com 新闻 内容 设计 阅读 方式 网站 www 用户 语言 python 创建 专利
Topic #5:
公司 领域 企业 互联网 中国 产业 智能 服务 人工智能 行业 平台 科技 市场 投资 创新 未来 金融 创业 工业 目前
Topic #6:
电子 应当 或者 案件 保护 规定 收集 是否 信用卡 法律 提取 申请 法院 通知 无法 相关 要求 记录 审查 检验
Topic #7:
用户 数据分析 客户 产品 公司 营销 研究 行为 医疗 数据挖掘 消费者 使用 银行 如何 网络 企业 案例 不同 商品 精准
Topic #8:
这个 就是 如果 可能 没有 很多 什么 他们 时候 但是 不是 自己 所以 一些 因为 这样 现在 非常 那么 已经
Topic #9:
中国 2016 增长 10 城市 市场 2015 大众 人口 30 关注 全国 同比 其中 20 12 00 行业 11 美国



？主题有先后之分吗？为什么可视化里面，有的圆圈大，有的圆圈小？

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)

D:\anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [29/Jun/2018 16:32:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 16:32:20] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 16:32:20] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 16:32:20] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 16:32:20] code 404, message Not Found
127.0.0.1 - - [29/Jun/2018 16:32:20] "GET /favicon.ico HTTP/1.1" 404 -


1. 如果只使用前两天语句，由于pyLDAvis包兼容性有些问题。因此在某些操作系统和软件环境下，执行了刚刚的语句后，没有报错，却也没有图形显示出来。所以可以添加后两句语句，这样会显示在一个新的标签页中
2. Jupyter会给你提示一些警告。不用管它。因为此时你的浏览器会弹出一个新的标签页，结果图形会在这个标签页里正确显示出来。
3. 如果你看完了图后，需要继续程序，就回到原先的标签页，点击Kernel菜单下的第一项Interrupt停止绘图，然后往下运行新的语句。
4. **图的左侧，用圆圈代表不同的主题，圆圈的大小代表了每个主题分别包含文章的数量。**
    + 图左侧的标识是  Marginal topic distribtion边际主题分布
5. **图的右侧，列出了最重要（频率最高）的30个关键词列表。注意当你没有把鼠标悬停在任何主题之上的时候，这30个关键词代表全部文本中提取到的30个最重要关键词。**
 + 图右侧的表示，蓝色代表全部文本里最高频率的词，红色代表所选主题的最高频率的词


6. 之前翻墙的话，只是前两个语句也可以运行，两种方法，都知道一下，没问题

观察可视化图发现，当主题数为10个时，编号为10的主题似乎与其他主题格格不入，其他的都抱团，就它一个自己呆着，观察发现，这个主题是法律类的，和其他科技类的肯定格格不入

另外，不论是5个还是10个主题，可能都不是最优的数量选择。你可以根据程序反馈的结果不断尝试。